# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from datetime import datetime

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F
import json
from sklearn.preprocessing import LabelEncoder

In [4]:
# main_data_dir =  "/media/mountHDD2"
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/thao/git/HeartResearch/Chapman Dataset/2D_model/Proposed_model
['Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/thao/git/HeartResearch/Chapman Dataset/2D_model/Proposed_model


In [11]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
label_file = data_dir + "/data_10_df.csv"

In [12]:
# data_df = pd.read_csv(data_dir + "/MUSE_20180120_121711_19000.csv")
label_df = pd.read_csv(label_file)
# label_df = label_df[['FileName', 'Rhythm']]

In [13]:
label_encoder = LabelEncoder()
label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])
print(label_df)

       Unnamed: 0                    FileName  Rhythm  New Label
0               0  MUSE_20180120_121711_19000       5          5
1               1  MUSE_20180120_121704_86000       5          5
2               2  MUSE_20180113_125357_13000       5          5
3               3  MUSE_20180113_134825_04000       7          7
4               4  MUSE_20180115_123455_79000       7          7
...           ...                         ...     ...        ...
10641       10641  MUSE_20181222_204246_47000      10         10
10642       10642  MUSE_20180115_120332_79000       5          5
10643       10643  MUSE_20180712_152507_30000       0          0
10644       10644  MUSE_20180118_181350_17000       5          5
10645       10645  MUSE_20180116_121646_28000       9          9

[10646 rows x 4 columns]


In [16]:
single_fns = label_df["FileName"].values.tolist()
single_mat_paths = [data_dir + f"/{x}.csv" for x in single_fns]

In [17]:
names = np.array(single_mat_paths)
label = np.array(label_df["New Label"].values)
print(type(label[0]))

<class 'numpy.int64'>


In [18]:
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=42)
splits = sss.split(names, label)

for tr_idxs, vl_idxs in splits:
    pass
    
tr_names, tr_label = names[tr_idxs].tolist(), label[tr_idxs].tolist()
vl_names, vl_label = names[vl_idxs].tolist(), label[vl_idxs].tolist()

tr_idx_0 = np.argwhere(np.array(tr_label) == 0).flatten().tolist()
tr_idx_1 = np.argwhere(np.array(tr_label) == 1).flatten().tolist()

vl_idx_0 = np.argwhere(np.array(vl_label) == 0).flatten().tolist()
vl_idx_1 = np.argwhere(np.array(vl_label) == 1).flatten().tolist()

batch_sampler = WeightedRandomSampler(weights=(0.8, 0.2), num_samples=len(tr_names))

assert len(tr_names) == len(tr_label)
assert len(vl_names) == len(vl_label)

print(f'[INFO]: Found {len(tr_names)} training samples, {len(vl_names)} validation samples')
print(f'[INFO]: {len(tr_idx_0)} 0s, {len(tr_idx_1)} 1s in train')
print(f'[INFO]: {len(vl_idx_0)} 0s, {len(vl_idx_1)} 1s in valid')

[INFO]: Found 10113 training samples, 533 validation samples
[INFO]: 423 0s, 1691 1s in train
[INFO]: 22 0s, 89 1s in valid


In [11]:
# data_dir = "/media/mountHDD2/khoibaocon"
# print(os.listdir(data_dir))
print(type(label_df["Rhythm"][0]))

<class 'numpy.int64'>


In [12]:
# main_df = pd.read_csv(data_dir + "/Label.csv")
# main_df.shape

In [13]:
# single_main_df = main_df[main_df["Second_label"].isnull()]
# single_fns = single_main_df["Recording"].values.tolist()
# single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]

In [14]:
ratio = [0.8, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

8516
8516


In [15]:
print(len(train_label))
print(len(valid_label))

8516
1065


In [16]:
label_df[label_df["FileName"] == filenames]["Rhythm"].values

array([5, 5, 5, ..., 0, 5, 9])

 # Data Loader

In [19]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [28]:
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 0:4]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        grayscale_images = (normalized_data * 255)
        grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        resized_images = F.interpolate(grayscale_images, size=(9*4,2500), mode='bilinear', align_corners=True)
        resized_images = resized_images.squeeze(0).squeeze(0)
        torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        torch_data_resize = self.transform(torch_data)

        # filename = data_path.split("/")[-1].split(".")[0]
        # label = self.label_df[self.label_df["File name"] == filename]["New Label"].values.item()
        label = self.label_df[idx]

        return torch_data_resize, label

    def __len__(self):
        return len(self.data_paths)    


In [29]:
train_ds = HeartData(tr_names, tr_label)
valid_ds = HeartData(vl_names, vl_label)

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    # shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [31]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.ori_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 11),
            # nn.Softmax(dim = 1)
        )
        
    def forward(self, x):        
        x = self.ori_model(x)
        return x
model = HeartModel()
x = torch.randn((1, 3, 224, 224))
y = torch.randint(0, 10, (1,))
out = model(x)

In [32]:
# model

# Training

In [33]:
epoch = 150
lr = 0.001 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0

model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))

In [34]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
loss_fn = nn.CrossEntropyLoss()
# checkpoint_folder = "run_efficientB0_heatmap_gamma0.5_lr0.0005"
# checkpoint_folder = "run_proposed_gamma0.3_0.01"

In [25]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = os.getcwd() + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [26]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [35]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

Epoch: 0


159it [00:28,  5.61it/s]

train loss: nan - train acc: 4.301394245031148



533it [00:04, 117.62it/s]

valid loss: nan - valid acc: 4.127579737335835
Epoch: 1



10it [00:02,  4.09it/s]
Exception ignored in: <function _ConnectionBase.__del__ at 0x7f441a3a9870>
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Exception ignored in: <function _ConnectionBase.__del__ at 0x7f441a3a9870>
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OS

KeyboardInterrupt: 

In [ ]:
# print(reports.keys())
# print(reports['macro avg'])

# Evaluation